# MLlib

źródło: https://spark.apache.org/docs/3.0.1/ml-guide.html

MLlib to biblioteka przeznaczona do uczenia maszynowego w Sparku. Jej celem jest prostego, skalowalnego rozwiązania w zakresie ML. Biblioteka zapewnia:

1. Algorytmy ML: klasyfikacja, regresja, klastrowanie i collaborative filtering
2. Featuryzacja: ekstrakcja zmiennych, transformacja, redukcja wymiarowości i selekcja
3. Pipelines: narzędzia do konstruowania, oceniania i dostrajania pipeline'ów ML
4. Trwałość: metody zapisywania i wczytywania algorytmów, modeli i pipeline'ów
5. Pozostałe narzędzia: algebra liniowa, statystyka, obsługa danych itp.

### Statystyki podstawowe

**Korelacja** </br>

Obliczanie korelacji między zmiennymi jest must - have. Na dzień dzisiejszy obsługiwane metody to korelacja Pearsona i Spearmana.

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
# Może chwilę potrwać
spark = SparkSession.builder.appName("UczymySięSparka").getOrCreate()

from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

c:\Users\Public\Anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Pearson correlation matrix:
DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])
Spearman correlation matrix:
DenseMatrix([[1.        , 0.10540926,        nan, 0.4       ],
             [0.10540926, 1.        ,        nan, 0.9486833 ],
             [       nan,        nan, 1.        ,        nan],
             [0.4       , 0.9486833 ,        nan, 1.        ]])


**Testowanie hipotez**
Testowanie hipotez jest bardzo ważne podczas określenia, czy wynik jest statystycznie istotny. Spark.ml obecnie obsługuje chi-kwadrat Pearsona (χ2).

In [3]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ["label", "features"])

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

pValues: [0.6872892787909721,0.6822703303362126]
degreesOfFreedom: [2, 3]
statistics: [0.75,1.5]


**Podsumowanie**
Spark udostępnia statystyki podsumowujące za pośrednictwem narzędzia Summarizer. Dostępne metryki to maks., min, średnia, suma, wariancja, std i liczba niezerowych w kolumnach.

In [5]:
from pyspark.ml.stat import Summarizer
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

df = spark.sparkContext.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
                     Row(weight=0.0, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()

# create summarizer for multiple metrics "mean" and "count"
summarizer = Summarizer.metrics("mean", "count")

# compute statistics for multiple metrics with weight
df.select(summarizer.summary(df.features, df.weight)).show(truncate=False)

# compute statistics for multiple metrics without weight
df.select(summarizer.summary(df.features)).show(truncate=False)

# compute statistics for single metric "mean" with weight
df.select(Summarizer.mean(df.features, df.weight)).show(truncate=False)

# compute statistics for single metric "mean" without weight
df.select(Summarizer.mean(df.features)).show(truncate=False)

+-----------------------------------+
|aggregate_metrics(features, weight)|
+-----------------------------------+
|{[1.0,1.0,1.0], 1}                 |
+-----------------------------------+

+--------------------------------+
|aggregate_metrics(features, 1.0)|
+--------------------------------+
|{[1.0,1.5,2.0], 2}              |
+--------------------------------+

+--------------+
|mean(features)|
+--------------+
|[1.0,1.0,1.0] |
+--------------+

+--------------+
|mean(features)|
+--------------+
|[1.0,1.5,2.0] |
+--------------+



### ML Pipelines ###
W tej sekcji przedstawiamy koncepcję pipeline'ów ML. Potoki ML zapewniają jednolity zestaw API zbudowanych w oparciu o ramki, które pomagają użytkownikom tworzyć i dostrajać pipeline'y.

**Główne pojęcia w Pipelines**
MLlib wystandaryzowało ML API żeby ułatwić łączenie wielu algorytmów w jeden pipeline lub worflow. W zaczynającej się teraz części warsztatu omówimy kluczowe koncepcje API Pipelines. Warto dodać, że API projektowane było na modłę projektu scikit-learn.

* **DataFrame**: to API wykorzystuje ramkę ze Spark SQL w charakterze danych pod modelowanie. Taka ramka może przechowywać różne typy danych np. tekst, wektory cech, empiryczne etykiety i prognozowane etykiety.

* **Transformator**: Transformator to algorytm, który może przekształcić jedną ramkę danych w inną ramkę danych. Np. model ML to Transformer, który przekształca DataFrame empiryczne w DataFrame z danymi prognostycznymi.

* **Estimator**: Estimator to algorytm, który można dopasować (fit) do ramki w celu stworzenia transformatora. Np. algorytm uczący się to estymator, który uczy się na ramce i tworzy model.

* **Pipeline**: łączy ze sobą wiele transformatorów i estymatorów, aby określić ML workflow.

* **Parametr**: Wszystkie transformatory i estymatory mają teraz wspólne API do tuningowania parametrów.

### Elementy pipeline'a ###
**Transformatory** </br>

Transformator to obiekt, który obejmuje zarówno transformatory featuresów jak i modele. Z technicznego punktu widzenia Transformer implementuje metodę transform(), która konwertuje jedną ramkę na inną, zazwyczaj poprzez dołączenie jednej lub więcej kolumn. Na przykład:

Transformator featuresa może pobrać ramkę, odczytać kolumnę (np. tekst), zmapować ją na nową kolumnę i zwrócić nową ramkę z dołączoną zmapowaną kolumną.
Model może pobrać ramkę, odczytać wektory cech, wykonać predykcję etykiety i zwrócić nową ramkę z prognozowanymi etykietami jako nową kolumnę ramki.

### Estymatory ###

Estymator to wynik algorytmu uczącego się. Z technicznego punktu widzenia Estimator implementuje metodę fit(), która przyjmuje jako argument ramkę i tworzy model, który jest Transformerem. Na przykład algorytm uczący się, taki jak LogisticRegression, jest estymatorem, a wywołanie funkcji fit() trenuje ten model, który dzięki temu staje się transformatorem.

### Pipeline ###
W ML powszechne jest uruchamianie sekwencji algorytmów w celu przetwarzania danych i uczenia modeli. Dla przykładu -  proces przetwarzania dokumentu tekstowego może obejmować kilka etapów:

* Podziel tekst każdego dokumentu na słowa.
* Konwertuj słowa każdego dokumentu na wektor numeryczny.
* Wyuczenie modelu przy użyciu wektorów cech i etykiet.

MLlib reprezentuje taki przepływ pracy jako Pipeline, który składa się z sekwencji PipelineStages (Transformers i Estimators) uruchamianych w określonej kolejności. W tej sekcji, jako przykladu, użyjemy prostego workflow.

### Działanie pipeline'a ###
Pipeline jest sekwencją etapów, a każdy etap jest albo Transformatorem, albo Estymatorem. Elementy uruchamiane są w określonej kolejności, a wejściowa ramka jest przekształcana podczas przechodzenia przez każdy z etapów. W przypadku etapu Transformer na ramce wywoływana jest metoda transform(). W przypadku etapu Estimator w celu stworzenia Transformera wywoływana jest metoda fit() (Transformer staje się wtedy częścią pipeline'u modelu lub estymatora), a metoda transform() jest wykonywana na ramce.

Poniżej prosta ilustracja:

Powyżej górny rząd przedstawia pipeline z trzema etapami. Pierwsze dwa (Tokenizer i HashingTF) to Transformers (niebieski), a trzeci (LogisticRegression) to Estimator (czerwony). Dolny wiersz reprezentuje dane przepływające przez pipeline, gdzie cylindry oznaczają ramki danych. Metoda Pipeline.fit() jest wywoływana na oryginalnej ramce, która zawiera nieprzetworzone dokumenty tekstowe i etykiety. Metoda Tokenizer.transform() dzieli nieprzetworzone dokumenty tekstowe na słowa, dodając nową kolumnę do ramki. Metoda HashingTF.transform() konwertuje kolumnę tekstową na wektor numeryczny, dodając nową kolumnę, zakodowaną kolumnę do ramki. Teraz, ponieważ LogisticRegression jest Estimatorem, Pipeline najpierw wywołuje LogisticRegression.fit() w celu utworzenia LogisticRegressionModel. Gdyby pipline miał więcej estymatorów, wywołałby metodę transform() LogisticRegressionModel na ramce przed przekazaniem jej do następnego etapu.

Pipeline jest estymatorem. Zatem po uruchomieniu metody fit() Pipeline tworzy PipelineModel, który jest Transformerem. Ten PipelineModel jest używany w czasie testu; poniższy rysunek to ilustruje:

Na powyższym rysunku PipelineModel ma taką samą liczbę etapów jak oryginalny Pipeline, ale wszystkie estymatory w oryginalnym Pipeline stały się Transformatorami. Kiedy metoda transform() obiektu PipelineModel jest wywoływana na testowym zbiorze danych, dane są przekazywane przez dopasowany (fitted) pipeline w określonej kolejności. Metoda transform() każdego etapu aktualizuje zestaw danych i przekazuje go do następnego etapu.

Pipelines i PipelineModels pomagają zagwarantować, że dane treningowe i testowe przechodzą przez identyczne etapy przetwarzania.

### Parametry ###

Estymatory i transformatory MLlib używają jednolitego API do określania parametrów.

Param to konkretny parametr z odpowiednią dokumentacją. ParamMap to zestaw par typu klucz - wartość.

Istnieją dwa główne sposoby przekazywania parametrów do algorytmu:

1. Ustaw parametry dla instancji. Przykładowo, jeśli lr jest instancją LogisticRegression, można wywołać lr.setMaxIter(10), aby lr.fit() użyła maksymalnie 10 iteracji. 
2. Przekaż ParamMap do fit() lub transform(). Wszelkie parametry w ParamMap zastąpią parametry określone wcześniej za pomocą setter methods.

Parametry należą do konkretnych instancji Estymatorów i Transformatorów. Na przykład, jeśli mamy dwie instancje LogisticRegression lr1 i lr2, możemy zbudować ParamMap z podanymi dwoma parametrami maxIter: ParamMap(lr1.maxIter -> 10, lr2.maxIter -> 20). Jest to przydatne, jeśli istnieją dwa algorytmy z parametrem maxIter w pipeline.

### Zapisywanie i ładowanie pipeline'ów ###
Często warto zapisać model lub pipeline na dysku. Od wersji Spark 2.3 API biblioteki spark.ml i pyspark.ml pokrywają oba te aspekty.

Zapisywanie modeli i pipeline'ów działa w Scali, Javie i Pythonie. R aktualnie używa zmodyfikowanego formatu, więc modele zapisane w R można wczytać z powrotem tylko w R.

### Poniżej przykłady: ### 
**Estymator, transformer i param**: 

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)
result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.myProbability, row.prediction))

**Przykład pipeline'u:**

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(4, spark i j k) --> prob=[0.6292098489668488,0.37079015103315116], prediction=0.000000
(5, l m n) --> prob=[0.984770006762304,0.015229993237696027], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.13412348342566147,0.8658765165743385], prediction=1.000000
(7, apache hadoop) --> prob=[0.9955732114398529,0.00442678856014711], prediction=0.000000


### Ekstrakcja, przekształcanie i wybieranie cech ###

Ta sekcja obejmuje popularne transformacje wektorów zmiennych. 

**Binaryzacja** </br>

Binaryzacja to proces mapowania cech liczbowych do cech binarnych (0/1). Binarizer przyjmuje wspólne parametry inputCol i outputCol, a także próg binaryzacji. Wartości funkcji większe niż próg są binaryzowane do 1; wartości równe lub mniejsze od progu są binaryzowane do 0. 

In [8]:
from pyspark.ml.feature import Binarizer

continuousDataFrame = spark.createDataFrame([
    (0, 0.1),
    (1, 0.8),
    (2, 0.2)
], ["id", "feature"])

binarizer = Binarizer(threshold=0.5, inputCol="feature", outputCol="binarized_feature")

binarizedDataFrame = binarizer.transform(continuousDataFrame)

print("Binarizer output with Threshold = %f" % binarizer.getThreshold())
binarizedDataFrame.show()

Binarizer output with Threshold = 0.500000
+---+-------+-----------------+
| id|feature|binarized_feature|
+---+-------+-----------------+
|  0|    0.1|              0.0|
|  1|    0.8|              1.0|
|  2|    0.2|              0.0|
+---+-------+-----------------+



**PCA** </br>

PCA to procedura statystyczna wykorzystująca transformację ortogonalną do przetransformowania zbioru potencjalnie skorelowanych zmiennych na zbiory nieskorelowanych cech statystycznych. Powstałe zbiory nazywane są głównymi składowymi. Poniższy przykład pokazuje, jak rzutować 5-wymiarowe wektory na 3-wymiarowe główne składowe.

In [9]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark.createDataFrame(data, ["features"])

pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)

+------------------------------------------------------------+
|pcaFeatures                                                 |
+------------------------------------------------------------+
|[1.6485728230883814,-4.0132827005162985,-1.0091435193998504]|
|[-4.645104331781533,-1.1167972663619048,-1.0091435193998501]|
|[-6.428880535676488,-5.337951427775359,-1.009143519399851]  |
+------------------------------------------------------------+



**StringIndexer** </br>

StringIndexer mapuje kolumnę etykiet na kolumnę indeksów etykiet. StringIndexer może kodować wiele kolumn. Indeksy zawierają się w przedziale [0, numLabels) i obsługiwane są cztery opcje szeregowania ich: 
* „frequencyDesc”: kolejność malejąca według częstotliwości etykiety (najczęstsza etykieta ma przypisywane 0),
* „frequencyAsc”: kolejność rosnąca według częstotliwości etykiety (najrzadziej występujaca etykieta ma przypisane 0), 
* „alphabetDesc”: malejąca kolejność alfabetyczna
* „alphabetAsc”: rosnąca kolejność alfabetyczna (domyślnie = „frequencyDesc”). 

Warto zapamiętać, że w przypadku tej samej częstotliwości w przypadku użycia „frequencyDesc”/”frequencyAsc”, ciągi są dalej sortowane według alfabetu.

In [10]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



**IndexToString** </br>

IndexToString mapuje kolumnę indeksów etykiet z powrotem do kolumny zawierającej oryginalne etykiety. Typowym przypadkiem użycia jest tworzenie indeksów z etykiet za pomocą StringIndexer, trenowanie modelu na tych indeksach i pobieranie oryginalnych etykiet za pomocą IndexToString. Można też wykorzystać własne etykiety.

In [11]:
from pyspark.ml.feature import IndexToString, StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = indexer.fit(df)
indexed = model.transform(df)

print("Transformed string column '%s' to indexed column '%s'"
      % (indexer.getInputCol(), indexer.getOutputCol()))
indexed.show()

print("StringIndexer will store labels in output column metadata\n")

converter = IndexToString(inputCol="categoryIndex", outputCol="originalCategory")
converted = converter.transform(indexed)

print("Transformed indexed column '%s' back to original string column '%s' using "
      "labels in metadata" % (converter.getInputCol(), converter.getOutputCol()))
converted.select("id", "categoryIndex", "originalCategory").show()

Transformed string column 'category' to indexed column 'categoryIndex'
+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+

StringIndexer will store labels in output column metadata

Transformed indexed column 'categoryIndex' back to original string column 'originalCategory' using labels in metadata
+---+-------------+----------------+
| id|categoryIndex|originalCategory|
+---+-------------+----------------+
|  0|          0.0|               a|
|  1|          2.0|               b|
|  2|          1.0|               c|
|  3|          0.0|               a|
|  4|          0.0|               a|
|  5|          1.0|               c|
+---+-------------+----------------+



**OneHotEncoder** </br>

Mając kategorie zamienione na odpowiadające im liczby możemy zamienić je także na kilka kolumn (ich liczba zależy od tego ile jest kategorii), które zawierają zera i jedynki oznaczające odpowiednio czy dany wiersz należy do kategorii czy nie. Metodę tę stosujemy, gdy używamy algorytmu, który może mieć problem ze zmiennymi liczbowymi (bo zakładają jakiś porządek).

In [12]:
from pyspark.ml.feature import OneHotEncoder

df = spark.createDataFrame([
    (0.0, 1.0),
    (1.0, 0.0),
    (2.0, 1.0),
    (0.0, 2.0),
    (0.0, 1.0),
    (2.0, 0.0)
], ["categoryIndex1", "categoryIndex2"])

encoder = OneHotEncoder(inputCols=["categoryIndex1", "categoryIndex2"],
                        outputCols=["categoryVec1", "categoryVec2"])
model = encoder.fit(df)
encoded = model.transform(df)
encoded.show()

+--------------+--------------+-------------+-------------+
|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2|
+--------------+--------------+-------------+-------------+
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           1.0|           0.0|(2,[1],[1.0])|(2,[0],[1.0])|
|           2.0|           1.0|    (2,[],[])|(2,[1],[1.0])|
|           0.0|           2.0|(2,[0],[1.0])|    (2,[],[])|
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           2.0|           0.0|    (2,[],[])|(2,[0],[1.0])|
+--------------+--------------+-------------+-------------+



**Efekt interakcji** </br>

Efekt interakcji to Transformer, który generuje pojedynczą kolumnę wektorową zawierającą iloczyn wszystkich kombinacji wartości z kolumn wejściowych.


In [13]:
from pyspark.ml.feature import Interaction, VectorAssembler

df = spark.createDataFrame(
    [(1, 1, 2, 3, 8, 4, 5),
     (2, 4, 3, 8, 7, 9, 8),
     (3, 6, 1, 9, 2, 3, 6),
     (4, 10, 8, 6, 9, 4, 5),
     (5, 9, 2, 7, 10, 7, 3),
     (6, 1, 1, 4, 2, 8, 4)],
    ["id1", "id2", "id3", "id4", "id5", "id6", "id7"])

assembler1 = VectorAssembler(inputCols=["id2", "id3", "id4"], outputCol="vec1")

assembled1 = assembler1.transform(df)

assembler2 = VectorAssembler(inputCols=["id5", "id6", "id7"], outputCol="vec2")

assembled2 = assembler2.transform(assembled1).select("id1", "vec1", "vec2")

interaction = Interaction(inputCols=["id1", "vec1", "vec2"], outputCol="interactedCol")

interacted = interaction.transform(assembled2)

interacted.show(truncate=False)

+---+--------------+--------------+------------------------------------------------------+
|id1|vec1          |vec2          |interactedCol                                         |
+---+--------------+--------------+------------------------------------------------------+
|1  |[1.0,2.0,3.0] |[8.0,4.0,5.0] |[8.0,4.0,5.0,16.0,8.0,10.0,24.0,12.0,15.0]            |
|2  |[4.0,3.0,8.0] |[7.0,9.0,8.0] |[56.0,72.0,64.0,42.0,54.0,48.0,112.0,144.0,128.0]     |
|3  |[6.0,1.0,9.0] |[2.0,3.0,6.0] |[36.0,54.0,108.0,6.0,9.0,18.0,54.0,81.0,162.0]        |
|4  |[10.0,8.0,6.0]|[9.0,4.0,5.0] |[360.0,160.0,200.0,288.0,128.0,160.0,216.0,96.0,120.0]|
|5  |[9.0,2.0,7.0] |[10.0,7.0,3.0]|[450.0,315.0,135.0,100.0,70.0,30.0,350.0,245.0,105.0] |
|6  |[1.0,1.0,4.0] |[2.0,8.0,4.0] |[12.0,48.0,24.0,12.0,48.0,24.0,48.0,192.0,96.0]       |
+---+--------------+--------------+------------------------------------------------------+



**StandardScaler** </br>

StandardScaler to Transformer, który przekształca zestaw wektorów tak, żeby miał średnią = 0 i odchylenie standardowe = 1. 

In [ ]:
from pyspark.ml.feature import StandardScaler

dataFrame = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(dataFrame)

# Normalize each feature to have unit standard deviation.
scaledData = scalerModel.transform(dataFrame)
scaledData.show()

**Bucketizer** </br>

Bucketizer przekształca kolumnę ciągłą w kolumnę kategoryczną. Wartości graniczne dla danej kategorii ustala użytkownik. 

In [15]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), -0.5, 0.0, 0.5, float("inf")]

data = [(-999.9,), (-0.5,), (-0.3,), (0.0,), (0.2,), (999.9,)]
dataFrame = spark.createDataFrame(data, ["features"])

bucketizer = Bucketizer(splits=splits, inputCol="features", outputCol="bucketedFeatures")

# Transform original data into its bucket index.
bucketedData = bucketizer.transform(dataFrame)

print("Bucketizer output with %d buckets" % (len(bucketizer.getSplits())-1))
bucketedData.show()

Bucketizer output with 4 buckets
+--------+----------------+
|features|bucketedFeatures|
+--------+----------------+
|  -999.9|             0.0|
|    -0.5|             1.0|
|    -0.3|             1.0|
|     0.0|             2.0|
|     0.2|             2.0|
|   999.9|             3.0|
+--------+----------------+



**ChiSqSelector**

ChiSqSelector oznacza wybór zmiennych na podstawie testu Chi-kwadrat. ChiSqSelector wykorzystuje test niezależności Chi-kwadrat, aby zdecydować, które zmienne wybrać. 

In [16]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])

selector = ChiSqSelector(numTopFeatures=1, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="clicked")

result = selector.fit(df).transform(df)

print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
result.show()

ChiSqSelector output with top 1 features selected
+---+------------------+-------+----------------+
| id|          features|clicked|selectedFeatures|
+---+------------------+-------+----------------+
|  7|[0.0,0.0,18.0,1.0]|    1.0|          [18.0]|
|  8|[0.0,1.0,12.0,0.0]|    0.0|          [12.0]|
|  9|[1.0,0.0,15.0,0.1]|    0.0|          [15.0]|
+---+------------------+-------+----------------+



**RFormula**

In [17]:
from pyspark.ml.feature import RFormula

dataset = spark.createDataFrame(
    [(7, "US", 18, 1.0),
     (8, "CA", 12, 0.0),
     (9, "NZ", 15, 0.0)],
    ["id", "country", "hour", "clicked"])

formula = RFormula(
    formula="clicked ~ country + hour",
    featuresCol="features",
    labelCol="label")

output = formula.fit(dataset).transform(dataset)
output.select("features", "label").show()

+--------------+-----+
|      features|label|
+--------------+-----+
|[0.0,0.0,18.0]|  1.0|
|[1.0,0.0,12.0]|  0.0|
|[0.0,1.0,15.0]|  0.0|
+--------------+-----+



### Modelowanie ###

**Regresja logistyczna** </br>

Używana, gdy zmienna zależna jest na skali dychotomicznej. Wartości zmiennej objaśnianej wskazują na wystąpienie, lub brak wystąpienia pewnego zdarzenia, które chcemy prognozować. Regresja logistyczna pozwala na obliczanie prawdopodobieństwa tego zdarzenia.



In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

**Gradient boosting** 

Gradient Boost jest algorytmem z rodziny metod zespołowych, a konkretnie boostingu. Metoda ta wykorzystuje wiele prostych algorytmów typu drzewo decyzyjne, aby zredukować zarówno bias i wariancje modelu. 

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

Inputy/Outputy modelu: https://spark.apache.org/docs/3.0.1/ml-classification-regression.html#gradient-boosted-trees-gbts

**Perceptron wielowarstwowy (ang. Multilayer Perceptron, MLP)**

Najpopularniejszy typ sieci neuronowych. Sieć tego typu składa się zwykle z jednej warstwy wejściowej, kilku warstw ukrytych oraz jednej warstwy wyjściowej. Ustalenie właściwej liczby warstw ukrytych oraz liczby neuronów znajdujących się w poszczególnych warstwach jest zadaniem dla zespołu budującego sieć. Warstwa wyjściowa może składać się z neuronów liniowych (w przypadku regresji) lub neuronów nieliniowych (w przypadku klasyfikacji). Trenowanuje się ją przy użyciu propagacji wstecznej

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm")\
    .load("data/mllib/sample_multiclass_classification_data.txt")

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [4, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

**Naiwny klasyfikator bayesowski**

Prosty klasyfikator probabilistyczny. Naiwne klasyfikatory bayesowskie są oparte na założeniu o wzajemnej niezależności predyktorów. Często nie mają one żadnego związku z rzeczywistością i właśnie z tego powodu nazywa się je naiwnymi. Bardziej opisowe jest określenie – „model cech niezależnych”. Ponadto model prawdopodobieństwa można wyprowadzić korzystając z twierdzenia Bayesa.

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm") \
    .load("data/mllib/sample_libsvm_data.txt")

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

**Generalized linear regression**

Uogólniony model liniowy rozszerza ogólny model liniowy w taki sposób, że zmienna zależna jest liniowo powiązana z wektorami cech za pośrednictwem określonej funkcji wiążącej. Model przyjmuje zmienną zależną, które nie ma rozkładu normalnego. Dzięki bardzo ogólnej postaci modelu obejmuje on wiele modeli statystycznych, takich jak regresja liniowa dla odpowiedzi o rozkładzie normalnym, modele logistyczne dla danych binarnych, modele logarytmiczno-liniowe dla danych o liczebności i wiele innych modeli statystycznych.

In [ ]:
from pyspark.ml.regression import GeneralizedLinearRegression

# Load training data
dataset = spark.read.format("libsvm")\
    .load("data/mllib/sample_linear_regression_data.txt")

glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)

# Fit the model
model = glr.fit(dataset)

# Print the coefficients and intercept for generalized linear regression model
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

# Summarize the model over the training set and print out some metrics
summary = model.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

### ML Tuning: tuning hiperparametrów i selekcja modelu ###

W tej sekcji opisano jak używać narzędzi MLlib do tuningowania algorytmów i pipeline'ów ML. Umożliwiają to dostępne w bibliotece funkcje Cross-Validation i inne.

**Wybór modelu (inaczej dostrajanie hiperparametrów)**

Ważnym zadaniem w ML jest wybór modelu. Nazywa się to również dostrajaniem. Tuning można przeprowadzić dla poszczególnych estymatorów, takich jak LogisticRegression, lub dla całych pipeline'ów, które obejmują wiele algorytmów, featuryzację i inne. Użytkownicy mogą dostroić cały pipeline jednocześnie zamiast dostrajać każdy element osobno.

MLlib pozwala zdecydować o wyborze modelu za pomocą narzędzi, takich jak CrossValidator i TrainValidationSplit. Wymagają one następujących elementów:

* Estimator: algorytm lub Pipeline do dostrojenia
* Zestaw ParamMaps: parametry do wyboru, czasami nazywane „siatką parametrów” do przeszukania
* Ewaluator: miara do zbadania, jak dobrze dopasowany Model radzi sobie z danymi testowymi

Ewaluatorem może być RegressionEvaluator dla zagadnień regresyjnych, BinaryClassificationEvaluator dla modeli binarnych, MulticlassClassificationEvaluator dla modeli wieloklasowych lub MultilabelClassificationEvaluator dla klasyfikacji z wieloma etykietami. Domyślna metryka używana do wyboru najlepszej pary ParamMap może zostać zastąpiona przez metodę setMetricName w każdym z tych ewaluatorów.

ParamGridBuilder może pomóc w konstrukcji siatki parametrów. Domyślnie zestawy parametrów z siatki są oceniane szeregowo. Ocenę parametrów można przeprowadzić równolegle, ustawiając partycjonowanie na wartość 2 lub większą (wartość 1 wykona obliczenia szeregowo) przed rozpoczęciem wyboru modelu za pomocą narzędzia CrossValidator lub TrainValidationSplit. Wartość dla paremetru określającego partycjonowanie należy wybierać ostrożnie, aby zmaksymalizować możliwości bez przekraczania zasobów klastra - wyższe wartości nie zawsze prowadzą do poprawy wydajności.

**Walidacja krzyżowa** </br>
CrossValidator rozpoczyna pracę od podzielenia zestawu danych na mniejsze zbiory, które są używane jako oddzielne zestawy danych treningowych i testowych. Np. przy k=3, CrossValidator wygeneruje 3 pary zestawów danych (trening, test), z których każdy wykorzystuje 2/3 danych do treningu i 1/3 do testowania. Aby ocenić konkretne wartości parametrów, CrossValidator oblicza średnią metrykę oceny dla tych 3 modeli. Po zidentyfikowaniu najlepszej pary dla ParamMap, CrossValidator ponownie dopasowuje Estimator przy użyciu najlepszej ParamMap używając całego zestawu danych. Poniżej przykład:

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Prepare training documents, which are labeled.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0),
    (4, "b spark who", 1.0),
    (5, "g d a y", 0.0),
    (6, "spark fly", 1.0),
    (7, "was mapreduce", 0.0),
    (8, "e spark program", 1.0),
    (9, "a e c l", 0.0),
    (10, "spark compile", 1.0),
    (11, "hadoop software", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# We now treat the Pipeline as an Estimator, wrapping it in a CrossValidator instance.
# This will allow us to jointly choose parameters for all Pipeline stages.
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# With 3 values for hashingTF.numFeatures and 2 values for lr.regParam,
# this grid will have 3 x 2 = 6 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=2)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)

# Prepare test documents, which are unlabeled.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "mapreduce spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    print(row)

Row(id=4, text='spark i j k', probability=DenseVector([0.2665, 0.7335]), prediction=1.0)
Row(id=5, text='l m n', probability=DenseVector([0.9204, 0.0796]), prediction=0.0)
Row(id=6, text='mapreduce spark', probability=DenseVector([0.4438, 0.5562]), prediction=1.0)
Row(id=7, text='apache hadoop', probability=DenseVector([0.8587, 0.1413]), prediction=0.0)


**Train - test split** </br>
Spark oferuje także TrainValidationSplit do dostrajania hiperparametrów. TrainValidationSplit ocenia każdą kombinację parametrów tylko raz. Jest tańszy, ale nie da tak wiarygodnych wyników, gdy zestaw danych szkoleniowych nie jest wystarczająco duży.

W przeciwieństwie do CrossValidator, TrainValidationSplit tworzy pojedynczą parę (treningową, testową) zestawu danych. Dzieli zestaw danych na dwie części za pomocą parametru trainRatio. Na przykład z trainRatio=0,75, TrainValidationSplit wygeneruje parę treningową i testową, w której 75% danych jest używanych do szkolenia, a 25% do walidacji.

Podobnie jak CrossValidator, TrainValidationSplit w końcu dopasuje Estimatorem model przy użyciu najlepszej mapy ParamMap i całego zestawu danych. Poniżej przykład:

In [24]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# Prepare training and test data.
data = spark.read.format("libsvm")\
    .load("data/mllib/sample_linear_regression_data.txt")
train, test = data.randomSplit([0.9, 0.1], seed=12345)

lr = LinearRegression(maxIter=10)

# We use a ParamGridBuilder to construct a grid of parameters to search over.
# TrainValidationSplit will try all combinations of values and determine best model using
# the evaluator.
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
model = tvs.fit(train)

# Make predictions on test data. model is the model with combination of parameters
# that performed best.
model.transform(test)\
    .select("features", "label", "prediction")\
    .show()

AnalysisException: Path does not exist: file:/c:/Users/Marta Głowińska/Desktop/DJL PySpark/warsztaty/data/mllib/sample_linear_regression_data.txt